<a href="https://colab.research.google.com/github/eungyu321/NVIDIA/blob/main/nvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# YOLOv8x 모델 로드
model = YOLO('yolov8x.pt')


In [21]:
# 이미지 로드 및 객체 검출
img = cv2.imread('/content/1.jpg')
results = model.predict(img, conf=0.5)  # 신뢰도 50% 이상 결과만 필터링

# 사람(person)과 우산(umbrella) 클래스 ID 확인
person_id = list(model.names.keys())[list(model.names.values()).index('person')]
umbrella_id = list(model.names.keys())[list(model.names.values()).index('umbrella')]

# 사람과 우산 객체 매칭
person_with_umbrella = 0
for result in results:
    persons = [box for box in result.boxes if int(box.cls) == person_id]
    umbrellas = [box for box in result.boxes if int(box.cls) == umbrella_id]

    # 우산과 사람의 바운딩 박스 중심점 비교
    for p in persons:
        px, py = p.xywh[0][0].item(), p.xywh[0][1].item()  # 사람 중심 좌표
        for u in umbrellas:
            ux, uy = u.xywh[0][0].item(), u.xywh[0][1].item()
            # 중심점 간 거리 계산 (픽셀 단위)
            distance = ((px-ux)**2 + (py-uy)**2)**0.5
            if distance < 100:  # 거리 임계값 설정
                person_with_umbrella += 1
                break

print(f"우산을 든 사람 수: {person_with_umbrella}")


0: 384x640 5 persons, 4 umbrellas, 2 handbags, 2828.9ms
Speed: 3.9ms preprocess, 2828.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
우산을 든 사람 수: 1


In [4]:
# YOLOv8 설치
!pip install ultralytics

# 기타 필요한 라이브러리
!pip install yt-dlp opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.3/977.3 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 861.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [5]:
import yt_dlp

# 유튜브 영상 다운로드 옵션
ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4',
}

# 유튜브 링크에서 영상 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=x9M5L2_h3wk'])


[youtube] Extracting URL: https://www.youtube.com/watch?v=x9M5L2_h3wk
[youtube] x9M5L2_h3wk: Downloading webpage
[youtube] x9M5L2_h3wk: Downloading tv client config
[youtube] x9M5L2_h3wk: Downloading player 73381ccc-main
[youtube] x9M5L2_h3wk: Downloading tv player API JSON
[youtube] x9M5L2_h3wk: Downloading ios player API JSON
[youtube] x9M5L2_h3wk: Downloading m3u8 information
[info] x9M5L2_h3wk: Downloading 1 format(s): 18
[download] Destination: video.mp4
[download] 100% of    2.52MiB in 00:00:01 at 2.06MiB/s   


In [6]:
from ultralytics import YOLO
import cv2
import os

# YOLOv8x 모델 로드
model = YOLO('yolov8x.pt')

# 자동차 클래스 번호 (COCO 클래스 기준)
CAR_CLASS_IDS = [2, 3, 5, 7]  # 자동차, 오토바이, 버스, 트럭

# 비디오 열기
cap = cv2.VideoCapture('video.mp4')
frame_count = 0
car_counts = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임당 1장씩 분석 (간단하게)
    frame_count += 1
    if frame_count % 5 != 0:
        continue

    # 객체 감지 실행
    results = model.predict(source=frame, conf=0.5, classes=CAR_CLASS_IDS, verbose=False)

    # 결과에서 자동차 수 세기
    boxes = results[0].boxes
    car_count = len(boxes)
    car_counts.append(car_count)

    # 출력 (필요시 시각화)
    print(f'Frame {frame_count}: {car_count} cars')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 131M/131M [00:00<00:00, 202MB/s]


Frame 5: 0 cars
Frame 10: 0 cars
Frame 15: 0 cars
Frame 20: 0 cars
Frame 25: 0 cars
Frame 30: 0 cars
Frame 35: 0 cars
Frame 40: 0 cars
Frame 45: 0 cars
Frame 50: 0 cars
Frame 55: 0 cars
Frame 60: 0 cars
Frame 65: 0 cars
Frame 70: 0 cars
Frame 75: 0 cars
Frame 80: 0 cars
Frame 85: 0 cars
Frame 90: 0 cars
Frame 95: 0 cars
Frame 100: 0 cars
Frame 105: 0 cars
Frame 110: 0 cars
Frame 115: 0 cars
Frame 120: 0 cars
Frame 125: 0 cars
Frame 130: 0 cars
Frame 135: 0 cars
Frame 140: 0 cars
Frame 145: 0 cars
Frame 150: 0 cars
Frame 155: 0 cars
Frame 160: 14 cars
Frame 165: 13 cars
Frame 170: 13 cars
Frame 175: 13 cars
Frame 180: 13 cars
Frame 185: 13 cars
Frame 190: 12 cars
Frame 195: 13 cars
Frame 200: 14 cars
Frame 205: 17 cars
Frame 210: 15 cars
Frame 215: 14 cars
Frame 220: 17 cars
Frame 225: 5 cars
Frame 230: 7 cars
Frame 235: 8 cars
Frame 240: 8 cars
Frame 245: 10 cars
Frame 250: 11 cars
Frame 255: 8 cars
Frame 260: 10 cars
Frame 265: 8 cars
Frame 270: 5 cars
Frame 275: 6 cars
Frame 280: 7 c

In [7]:
# YOLOv8 설치
!pip install ultralytics

# 유튜브 영상 다운로드 및 OpenCV 설치
!pip install yt-dlp opencv-python-headless

In [8]:
import yt_dlp

# 유튜브 영상 다운로드 옵션
ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4',
}

# 유튜브에서 영상 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=x9M5L2_h3wk'])

[youtube] Extracting URL: https://www.youtube.com/watch?v=x9M5L2_h3wk
[youtube] x9M5L2_h3wk: Downloading webpage
[youtube] x9M5L2_h3wk: Downloading tv client config
[youtube] x9M5L2_h3wk: Downloading player 73381ccc-main
[youtube] x9M5L2_h3wk: Downloading tv player API JSON
[youtube] x9M5L2_h3wk: Downloading ios player API JSON
[youtube] x9M5L2_h3wk: Downloading m3u8 information
[info] x9M5L2_h3wk: Downloading 1 format(s): 18
[download] video.mp4 has already been downloaded
[download] 100% of    2.52MiB


In [9]:
import cv2
from ultralytics import YOLO

# YOLOv8x 모델 로드
model = YOLO('yolov8x.pt')

# 자동차 관련 클래스 ID (COCO 데이터셋 기준)
CAR_CLASS_IDS = [2, 3, 5, 7]  # 자동차, 오토바이, 버스, 트럭

# 원본 영상 불러오기
cap = cv2.VideoCapture('video.mp4')

# 저장할 영상 설정 (코덱: MP4V, FPS: 원본과 동일하게 유지)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8로 객체 감지 실행
    results = model(frame)

    # 결과에서 감지된 객체들을 가져오기
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            class_id = int(box.cls[0])  # 클래스 ID

            # 자동차, 버스, 트럭 등만 표시
            if class_id in CAR_CLASS_IDS:
                confidence = box.conf[0]  # 신뢰도
                label = f"{model.names[class_id]} {confidence:.2f}"

                # 박스 및 텍스트 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 프레임 저장
    out.write(frame)

# 리소스 해제
cap.release()
out.release()

print("🎥 자동차 감지된 영상 저장 완료: output.mp4")


0: 384x640 1 person, 1 tie, 58.8ms
Speed: 1.4ms preprocess, 58.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 38.4ms
Speed: 2.0ms preprocess, 38.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 38.0ms
Speed: 1.4ms preprocess, 38.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 38.0ms
Speed: 1.7ms preprocess, 38.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 38.0ms
Speed: 1.8ms preprocess, 38.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 38.0ms
Speed: 1.7ms preprocess, 38.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 32.6ms
Speed: 2.4ms preprocess, 32.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 32.2ms
Speed: 2.0ms preprocess, 32.2ms i

In [11]:
# YOLOv8 설치
!pip install ultralytics

# 유튜브 영상 다운로드 및 OpenCV 설치
!pip install yt-dlp opencv-python-headless

In [12]:
import yt_dlp

# 유튜브 영상 다운로드 옵션
ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4',
}

# 유튜브에서 영상 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=x9M5L2_h3wk'])

[youtube] Extracting URL: https://www.youtube.com/watch?v=x9M5L2_h3wk
[youtube] x9M5L2_h3wk: Downloading webpage
[youtube] x9M5L2_h3wk: Downloading tv client config
[youtube] x9M5L2_h3wk: Downloading player 73381ccc-main
[youtube] x9M5L2_h3wk: Downloading tv player API JSON
[youtube] x9M5L2_h3wk: Downloading ios player API JSON
[youtube] x9M5L2_h3wk: Downloading m3u8 information
[info] x9M5L2_h3wk: Downloading 1 format(s): 18
[download] video.mp4 has already been downloaded
[download] 100% of    2.52MiB


In [13]:
import cv2
from ultralytics import YOLO

# YOLOv8x 모델 로드
model = YOLO('yolov8x.pt')

# 자동차 관련 클래스 ID (COCO 데이터셋 기준)
CAR_CLASS_IDS = [2, 3, 5, 7]  # 자동차, 오토바이, 버스, 트럭

# 원본 영상 불러오기
cap = cv2.VideoCapture('video.mp4')

# 저장할 영상 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

frame_count = 0
car_counts = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # YOLOv8로 객체 감지 실행
    results = model(frame)

    car_count = 0  # 현재 프레임의 자동차 개수

    # 결과에서 감지된 객체 가져오기
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            class_id = int(box.cls[0])  # 클래스 ID

            # 자동차, 버스, 트럭 등만 감지
            if class_id in CAR_CLASS_IDS:
                confidence = box.conf[0]  # 신뢰도
                label = f"{model.names[class_id]} {confidence:.2f}"
                car_count += 1

                # 박스 및 텍스트 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 감지된 자동차 개수 저장
    car_counts.append(car_count)

    # 자동차 개수를 화면에 표시
    cv2.putText(frame, f"Cars: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 프레임 저장
    out.write(frame)

cap.release()
out.release()

print("🎥 자동차 감지된 영상 저장 완료: output.mp4")


0: 384x640 1 person, 1 tie, 52.4ms
Speed: 1.9ms preprocess, 52.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 46.4ms
Speed: 1.6ms preprocess, 46.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 46.4ms
Speed: 1.7ms preprocess, 46.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 43.1ms
Speed: 1.5ms preprocess, 43.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 33.0ms
Speed: 1.8ms preprocess, 33.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 33.2ms
Speed: 1.7ms preprocess, 33.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 32.6ms
Speed: 1.6ms preprocess, 32.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 32.3ms
Speed: 1.5ms preprocess, 32.3ms i

In [14]:
from google.colab import files
files.download('output.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# YOLOv11 설치
!pip install ultralytics

# 유튜브 영상 다운로드 및 OpenCV 설치
!pip install yt-dlp opencv-python-headless

In [21]:
import yt_dlp

# 유튜브 영상 다운로드 옵션
ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4',
}

# 유튜브에서 영상 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=x9M5L2_h3wk'])

[youtube] Extracting URL: https://www.youtube.com/watch?v=x9M5L2_h3wk
[youtube] x9M5L2_h3wk: Downloading webpage
[youtube] x9M5L2_h3wk: Downloading tv client config
[youtube] x9M5L2_h3wk: Downloading player 73381ccc-main
[youtube] x9M5L2_h3wk: Downloading tv player API JSON
[youtube] x9M5L2_h3wk: Downloading ios player API JSON
[youtube] x9M5L2_h3wk: Downloading m3u8 information
[info] x9M5L2_h3wk: Downloading 1 format(s): 18
[download] video.mp4 has already been downloaded
[download] 100% of    2.52MiB


In [25]:
import cv2
from ultralytics import YOLO

# YOLOv11 모델 로드
model = yolo('yolo11.pt')

# 자동차 관련 클래스 ID (COCO 데이터셋 기준)
CAR_CLASS_IDS = [2, 3, 5, 7]  # 자동차, 오토바이, 버스, 트럭

# 원본 영상 불러오기
cap = cv2.VideoCapture('video.mp4')

# 저장할 영상 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

frame_count = 0
car_counts = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # YOLOv11로 객체 감지 실행
    results = model(frame)

    car_count = 0  # 현재 프레임의 자동차 개수

    # 결과에서 감지된 객체 가져오기
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            class_id = int(box.cls[0])  # 클래스 ID

            # 자동차, 버스, 트럭 등만 감지
            if class_id in CAR_CLASS_IDS:
                confidence = box.conf[0]  # 신뢰도
                label = f"{model.names[class_id]} {confidence:.2f}"
                car_count += 1

                # 박스 및 텍스트 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 감지된 자동차 개수 저장
    car_counts.append(car_count)

    # 자동차 개수를 화면에 표시
    cv2.putText(frame, f"Cars: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 프레임 저장
    out.write(frame)

cap.release()
out.release()

print("🎥 자동차 감지된 영상 저장 완료: output.mp4")

NameError: name 'yolo' is not defined

In [26]:
# YOLOv11 설치
!pip install ultralytics

# 유튜브 영상 다운로드 및 OpenCV 설치
!pip install yt-dlp opencv-python-headless

In [27]:
import yt_dlp

# 유튜브 영상 다운로드 옵션
ydl_opts = {
    'format': 'best',
    'outtmpl': 'video.mp4',
}

# 유튜브에서 영상 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=x9M5L2_h3wk'])

[youtube] Extracting URL: https://www.youtube.com/watch?v=x9M5L2_h3wk
[youtube] x9M5L2_h3wk: Downloading webpage
[youtube] x9M5L2_h3wk: Downloading tv client config
[youtube] x9M5L2_h3wk: Downloading player 73381ccc-main
[youtube] x9M5L2_h3wk: Downloading tv player API JSON
[youtube] x9M5L2_h3wk: Downloading ios player API JSON
[youtube] x9M5L2_h3wk: Downloading m3u8 information
[info] x9M5L2_h3wk: Downloading 1 format(s): 18
[download] video.mp4 has already been downloaded
[download] 100% of    2.52MiB


In [32]:
import cv2
from ultralytics import YOLO

# YOLOv11 모델 로드
model = YOLO('YOLOv11.pt')

# 자동차 관련 클래스 ID (COCO 데이터셋 기준)
CAR_CLASS_IDS = [2, 3, 5, 7]  # 자동차, 오토바이, 버스, 트럭

# 원본 영상 불러오기
cap = cv2.VideoCapture('video.mp4')

# 저장할 영상 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

frame_count = 0
car_counts = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # YOLOv11로 객체 감지 실행
    results = model(frame)

    car_count = 0  # 현재 프레임의 자동차 개수

    # 결과에서 감지된 객체 가져오기
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
            class_id = int(box.cls[0])  # 클래스 ID

            # 자동차, 버스, 트럭 등만 감지
            if class_id in CAR_CLASS_IDS:
                confidence = box.conf[0]  # 신뢰도
                label = f"{model.names[class_id]} {confidence:.2f}"
                car_count += 1

                # 박스 및 텍스트 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 감지된 자동차 개수 저장
    car_counts.append(car_count)

    # 자동차 개수를 화면에 표시
    cv2.putText(frame, f"Cars: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 프레임 저장
    out.write(frame)

cap.release()
out.release()

print("🎥 자동차 감지된 영상 저장 완료: output.mp4")

FileNotFoundError: [Errno 2] No such file or directory: 'YOLOv11.pt'

In [33]:
# 필수 패키지 설치 (Colab에서 한 번만 실행)
!pip install ultralytics opencv-python
!apt-get update && apt-get install -y libgl1-mesa-glx

from ultralytics import YOLO
import cv2
import numpy as np
from google.colab import drive

# 구글 드라이브 마운트 (영상 저장용)
drive.mount('/content/drive')

# YOLOv8x 모델 로드 (자동 다운로드)
model = YOLO('yolov8x.pt')

# 비디오 처리 함수
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # 출력 영상 설정
    output_path = '/content/drive/MyDrive/output_annotated.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    total_cars = 0
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 객체 검출 (GPU 가속)
        results = model.predict(frame, conf=0.5, device='cuda')

        # 프레임별 자동차 수 계산
        car_count = 0
        for result in results:
            for box in result.boxes:
                if int(box.cls) == 2:  # COCO 클래스 2: car
                    car_count += 1

        total_cars += car_count
        frame_count += 1

        # 화면에 차량 수 표시
        annotated_frame = results[0].plot()
        cv2.putText(annotated_frame, f'Cars: {car_count}', (10, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # 영상 저장
        out.write(annotated_frame)

    cap.release()
    out.release()
    print(f"총 평균 차량 수: {total_cars/frame_count:.2f}")
    return output_path

# 유튜브 영상 다운로드 (pytube 설치 필요)
!pip install pytube
from pytube import YouTube

yt = YouTube('https://www.youtube.com/watch?v=x9M5L2_h3wk')
video_stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
video_path = video_stream.download(filename='input_video.mp4')

# 영상 처리 실행
output_video = process_video(video_path)
print(f"주석 처리된 영상 저장 경로: {output_video}")

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,978 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,775 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,241 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]
Get:13 https://develo

HTTPError: HTTP Error 400: Bad Request

In [34]:
# Ultralytics 패키지 설치
!pip install ultralytics

# 유튜브 영상 다운로드를 위한 라이브러리 설치
!pip install yt-dlp

# OpenCV 설치
!pip install opencv-python-headless

In [35]:
from ultralytics import YOLO

# YOLOv11 모델 로드
model = YOLO('yolo11x.pt')  # 'yolo11x.pt'는 가장 큰 크기의 YOLOv11 모델입니다.

100%|██████████| 109M/109M [00:01<00:00, 65.9MB/s]


In [36]:
import cv2
import yt_dlp
import os

# 유튜브 영상 URL
video_url = 'https://www.youtube.com/watch?v=x9M5L2_h3wk'

# yt-dlp를 사용하여 영상 다운로드
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'outtmpl': 'video.mp4',
    'merge_output_format': 'mp4',
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 다운로드한 영상 열기
cap = cv2.VideoCapture('video.mp4')

# 저장할 영상 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# 클래스 ID 설정 (자동차 관련 클래스)
car_class_ids = [2, 3, 5, 7]  # COCO 데이터셋 기준: 2=자동차, 3=오토바이, 5=버스, 7=트럭

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv11 모델을 사용하여 객체 감지
    results = model(frame)

    car_count = 0
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            if class_id in car_class_ids:
                car_count += 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                label = f"{model.names[class_id]} {confidence:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 프레임에 자동차 개수 표시
    cv2.putText(frame, f"Cars: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 결과 프레임 저장
    out.write(frame)

cap.release()
out.release()

print("자동차 감지 및 개수 세기가 완료되었습니다. 결과 영상은 'output.mp4'에 저장되었습니다.")

[youtube] Extracting URL: https://www.youtube.com/watch?v=x9M5L2_h3wk
[youtube] x9M5L2_h3wk: Downloading webpage
[youtube] x9M5L2_h3wk: Downloading tv client config
[youtube] x9M5L2_h3wk: Downloading player 73381ccc-main
[youtube] x9M5L2_h3wk: Downloading tv player API JSON
[youtube] x9M5L2_h3wk: Downloading ios player API JSON
[youtube] x9M5L2_h3wk: Downloading m3u8 information
[info] x9M5L2_h3wk: Downloading 1 format(s): 137+251
[download] video.mp4 has already been downloaded

0: 384x640 1 person, 1 tie, 118.0ms
Speed: 2.1ms preprocess, 118.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 48.4ms
Speed: 1.9ms preprocess, 48.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 46.4ms
Speed: 1.8ms preprocess, 46.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 37.4ms
Speed: 1.9ms preprocess, 37.4ms inference, 2.1ms postprocess per image at shape

In [37]:
from google.colab import files
files.download('output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
# 1️⃣ 필수 라이브러리 설치
!apt-get install -y libgl1-mesa-glx
!pip install -U yt-dlp ultralytics opencv-python

import yt_dlp
import cv2
import time
import os
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# 2️⃣ YouTube 영상 다운로드 (yt-dlp 사용)
def download_youtube_video(url, output_path="traffic.mp4"):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best',
        'outtmpl': output_path,  # 저장할 파일 경로
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"✅ YouTube 영상 다운로드 완료: {output_path}")
    except Exception as e:
        print(f"❌ YouTube 다운로드 오류: {e}")

# 🎯 🔄 **새로운 YouTube 영상 URL 적용**
video_url = 'https://www.youtube.com/watch?v=QedNLyleW1w'  # 새로운 영상 URL
download_youtube_video(video_url, "/content/traffic.mp4")

# 3️⃣ YOLOv11x 모델 로드
model = YOLO('yolo11x.pt')  # 🔄 변경된 모델명

# 4️⃣ 차량 감지 및 카운팅
cap = cv2.VideoCapture("/content/traffic.mp4")

if not cap.isOpened():
    print("❌ 비디오 파일을 열 수 없습니다. 다운로드가 실패했을 가능성이 있습니다.")
else:
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)  # FPS 정보 가져오기
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 총 프레임 수
    duration = total_frames / fps  # 전체 영상 길이(초)

    print(f"🎥 영상 정보: {duration:.2f}초, {fps:.2f} FPS, 총 {total_frames} 프레임")

    # ✅ MP4V 코덱 사용
    out = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), int(fps), (frame_width, frame_height))

    # VideoWriter 정상 작동 여부 확인
    if not out.isOpened():
        print("❌ VideoWriter가 정상적으로 열리지 않았습니다. 코덱을 확인하세요.")
    else:
        print("✅ VideoWriter 초기화 성공")

    car_ids = set()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 🚗 차량 감지 및 추적
        results = model.track(frame, persist=True, tracker="bytetrack.yaml", classes=[2,3,5,7], conf=0.5)

        if results and results[0].boxes.id is not None:
            for box_id in results[0].boxes.id.cpu().numpy().astype(int):
                car_ids.add(box_id)

        # 감지된 프레임 저장
        annotated_frame = results[0].plot() if results else frame
        out.write(annotated_frame)

    cap.release()
    out.release()

    # 5️⃣ 결과 출력
    print(f"🔹 전체 영상에서 통과한 차량 수: {len(car_ids)}대")
    print("\n📌 결과 영상 확인 방법:")
    print("1. 좌측 폴더 아이콘 📁 클릭")
    print("2. `/content/output.mp4` 파일 우클릭 → [다운로드] 또는 [미리보기]")

    # 6️⃣ output.mp4 파일 존재 여부 확인 후 다운로드
    output_path = "/content/output.mp4"
    if os.path.exists(output_path):
        print("✅ output.mp4 파일이 정상적으로 저장되었습니다.")
        from google.colab import files
        files.download(output_path)
    else:
        print("❌ output.mp4 파일이 생성되지 않았습니다. OpenCV 설정을 확인하세요.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
[youtube] Extracting URL: https://www.youtube.com/watch?v=QedNLyleW1w
[youtube] QedNLyleW1w: Downloading webpage
[youtube] QedNLyleW1w: Downloading tv client config
[youtube] QedNLyleW1w: Downloading player 73381ccc-main
[youtube] QedNLyleW1w: Downloading tv player API JSON
[youtube] QedNLyleW1w: Downloading ios player API JSON
[youtube] QedNLyleW1w: Downloading m3u8 information
[info] QedNLyleW1w: Downloading 1 format(s): 137+140
[download] Destination: /content/traffic.f137.mp4
[download] 100% of   19.30MiB in 00:00:01 at 11.13MiB/s  
[download] Destination: /content/traffic.f140.m4a
[download] 100% of  617.92KiB in 00:00:00 at 2.14MiB/s   
[Merger] Merging formats into "/content/traffic.mp4"
Deleting original file /content/traffic.f137.mp4 (pass -k to k

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>